# D&D encounter manager 

### Requirements
- Select enemies for an encounter if provided with a challenge rating and environment (I can upload possible options and it should randomly choose from a list of say 100)
- Roll 20-sided die and determine order in which players will take turns
- Quick access to monster stats and possible attacks/moves
- Manage HP for each monster and track turn count-downs where applicable
- Extra: LLM API to help generate interesting names, monster characteristics / traits, things they say, things they are wearing

In [210]:
import numpy as np
import random
import math
from operator import attrgetter
import warnings

## Functions

In [211]:
def roll_dice(sides = 20):
    """Returns a number between 1 and the sides of a dice using a random uniform distribution (default 20-sided)"""
    roll = np.random.randint(1, sides + 1)
    
    if sides not in {4, 6, 8, 10, 12, 20, 100}:
        warnings.warn(f"Warning: It is uncommon to roll a {sides}-sided die. Are you sure?")
        return roll
    else:
        return roll

def get_multiple_dice_rolls(roll_count, sides):
    """Returns a list of multiple dic rolls. Useful for rolls such as 2d6, 6d6, etc."""
    dice_rolls_list = []

    for roll in range(roll_count):
        dice_rolls_list.append(roll_dice(sides))
    
    return dice_rolls_list

def int_to_ordinal(n):
    """Convert an integer to its ordinal representation"""
    if not isinstance(n, int) or n < 1:
        raise ValueError("Input is not a positive integer.")

    if n% 100 in range(11, 14):
        suffix = "th"
    elif n % 10 in range(4, 10):
        suffix = "th"
    elif n % 10 == 0:
        suffix = "th"
    elif n % 10 == 1:
        suffix = "st"
    elif n % 10 == 2:
        suffix = "nd"
    elif n % 10 == 3:
        suffix = "rd"
    
    return f"{n}{suffix}"

def ability_score_to_modifier(ability_score):
    """Converts ability scores to ability score modifiers"""
    modifier = math.floor((ability_score - 10) / 2)
    
    return modifier

def roll_for_initiative(target):
    """Initiative roll is a 1d20 plus the dexterity modifier"""
    roll = roll_dice(20)
    roll_modified = roll + ability_score_to_modifier(target.abilityScores.dexterity)
    
    return roll_modified

def instantiate_weapon_from_dict(name):
    """Create an object instance using the Weapon class"""
    if name not in weapons_dict:
        raise ValueError("Weapon name does not exist.")
    else:
        weapon_stats = Weapon(
            name = name,
            cost = weapons_dict[name]["cost"],
            damageDiceCount = weapons_dict[name]["damage_dice_count"],
            damageDiceSides = weapons_dict[name]["damage_dice_sides"],
            weight = weapons_dict[name]["weight"],
            weaponType = weapons_dict[name]["weapon_type"]
            )
    
    return weapon_stats


## Weapons dictionary

In [212]:
# Source: 2014 Player's Handbook
weapons_dict = {
    "scimitar":
        {"cost": 25,
         "damage_dice_count": 1,
         "damage_dice_sides": 6,
         "weight": 3,
         "weapon_type": "melee"
         },
    "battle axe":
        {"cost": 10,
         "damage_dice_count": 1,
         "damage_dice_sides": 8,
         "weight": 4,
         "weapon_type": "melee"
         }
    }

## Classes used by both Player and Monster classes

In [213]:
class AbilityScores:
    """Ability scores for creatures"""
    def __init__(self, strength, dexterity, constitution, intelligence, wisdom, charisma):
        ability_scores_set = {strength, dexterity, constitution, intelligence, wisdom, charisma}

        if any(score > 30 for score in ability_scores_set):
            raise ValueError("Ability scores must be between 1 and 30.")
        elif any(score < 1 for score in ability_scores_set):
            raise ValueError("Ability scores must be between 1 and 30.")
        else:
            self.strength = strength
            self.dexterity = dexterity
            self.constitution = constitution
            self.intelligence = intelligence
            self.wisdom = wisdom
            self.charisma = charisma

class Alignment:
    """Alignment for creatures"""
    def __init__(self, lawfulness, morality):
        self.valid_lawfulness = {"lawful", "neutral", "chaotic"}
        self.valid_morality = {"good", "neutral", "evil"}

        if lawfulness in self.valid_lawfulness:
            self.lawfulness = lawfulness
        else:
            raise ValueError(f"Invalid lawfulness: {lawfulness}. Must be one of {self.valid_lawfulness}.")

        if morality in self.valid_morality:
            self.morality = morality
        else:
            raise ValueError(f"Invalid morality: {morality}. Must be one of {self.valid_morality}.")

class Attack:
    """Attacks for creatures"""
    def __init__(self, target, attackRoll, attackRollModified, damageRoll):
        self.target = target
        self.attackRoll = attackRoll
        self.attackRollModified = attackRollModified
        self.damageRoll = damageRoll

class GoldPieces:
    """Amount of gold"""
    def __init__(self, value):
        self.value = value


## Player class and Weapon class to be used by players only

In [214]:
class Player:
    """Playable hero"""
    def __init__(self, name, species, playerClass, abilityScores, proficiencyBonus, alignment, hitPointsMax, armorClass, weapons, inventory = [], size = "medium", speed = 30):
        self.name = name
        self.species = species
        self.playerClass = playerClass
        self.abilityScores = abilityScores
        self.proficiencyBonus = proficiencyBonus
        self.alignment = alignment
        self.hitPointsMax = hitPointsMax
        self.armorClass = armorClass
        self.weapons = weapons
        # Attributes below have default values which can be overriden
        self.inventory = inventory
        self.size = size
        self.speed = speed
        # Attributes below are set by default
        self.hitPoints = self.hitPointsMax
        self.initiativeRollModified = None
        self.initiativeRank = None
    
    def __str__(self):
        return f"{self.playerClass.capitalize()} {self.name} jumps into battle!"
    
    def attack(self, target, weapon):
        """Attacks a target creature"""
        if weapon not in self.weapons:
            raise ValueError(f'Player is not equipped with {weapon}.')
        
        attackRoll = roll_dice(20)
        
        if weapon.weaponType not in {"melee", "ranged"}:
            raise ValueError("Weapon type must be melee or ranged.")
        elif weapon.weaponType == "melee":
            attackAndDamageRollModifier = ability_score_to_modifier(self.abilityScores.strength)
        elif weapon.weaponType == "ranged":
            attackAndDamageRollModifier = ability_score_to_modifier(self.abilityScores.dexterity)
        
        attackDetails = Attack(
            target = target,
            attackRoll = attackRoll,
            attackRollModified = attackRoll + self.proficiencyBonus + attackAndDamageRollModifier,
            damageRoll = sum(get_multiple_dice_rolls(weapon.damageDiceCount, weapon.damageDiceSides)) + attackAndDamageRollModifier
            )
        
        return attackDetails
    
    def rollForInitiative(self):
        """Determine order of combat"""
        initiativeRollModified = roll_for_initiative(self)
        self.initiativeRollModified = initiativeRollModified
    
    def takeLoot(self, target):
        """Loots a target creature by adding all items to Player's inventory and removing it from target creature's inventory. Combines gold pieces together"""
        for item in target.inventory:
            self.inventory.append(item)

        target.inventory = []

        goldValueSum = 0

        for item in self.inventory:
            if isinstance(item, GoldPieces):
                goldValueSum = item.value + goldValueSum
                self.inventory.remove(item)
        
        self.inventory.append(GoldPieces(value = goldValueSum))

class Weapon:
    """Class for all weapons"""
    def __init__(self, name, cost, damageDiceCount, damageDiceSides, weight, weaponType):
        self.name = name
        self.cost = cost
        self.damageDiceCount = damageDiceCount
        self.damageDiceSides = damageDiceSides
        self.weight = weight
        self.weaponType = weaponType


## Monster classes

In [215]:
# Parent class
class Monster:
    """Parent monster class"""
    def __init__(self, name, size, creatureType, alignment, armorClass, hitPointsMax, speed, abilityScores, proficiencyBonus, challengeRating, inventory):
        self.name = name
        self.size = size
        self.creatureType = creatureType
        self.alignment = alignment
        self.armorClass = armorClass
        self.hitPointsMax = hitPointsMax
        self.speed = speed
        self.abilityScores =  abilityScores
        self.proficiencyBonus = proficiencyBonus
        self.challengeRating = challengeRating
        self.inventory = inventory
        # Attributes below are set by default
        self.hitPoints = self.hitPointsMax
        self.initiativeRoll = None
        self.initiativeRollModified = None
        self.initiativeRank = None
        self.weapons = dict()
    
    def equipWeapon(self, name, attackRollModifier, damageDiceCount, damageDiceSides, damageRollModifier):
        """Adds a weapon key to the monster's dictionary"""
        if name in self.weapons:
            raise ValueError("Weapon is already equipped.")

        self.weapons[name] = {"attackRollModifier": attackRollModifier, "damageDiceCount": damageDiceCount, "damageDiceSides": damageDiceSides, "damageRollModifier": damageRollModifier}

    def unequipWeapon(self, name):
        """Removes a weapon key from the monster's dictionary"""
        del self.weapons[name]

    def attack(self, target, weapon):
        """Attacks a target creature"""
        if weapon not in self.weapons:
            raise ValueError("Attacking monster does not have this weapon equipped.")
        
        attackRoll = roll_dice(20)
        attackRollModifier = self.weapons[weapon]["attackRollModifier"]
        damageDiceCount = self.weapons[weapon]["damageDiceCount"]
        damageDiceSides = self.weapons[weapon]["damageDiceSides"]
        damageRollModifier = self.weapons[weapon]["damageRollModifier"]
        
        attackDetails = Attack(
            target = target,
            attackRoll = attackRoll,
            attackRollModified = attackRoll + attackRollModifier,
            damageRoll = sum(get_multiple_dice_rolls(damageDiceCount, damageDiceSides)) + damageRollModifier
            )
        return attackDetails
    
    def rollForInitiative(self):
        """Determine order of combat"""
        initiativeRollModified = roll_for_initiative(self)
        self.initiativeRollModified = initiativeRollModified

# Children classes
class Goblin(Monster):
    """Goblin creature (2014 Monster Manual)"""
    def __init__(self, name):
        super().__init__(
            name,
            size = "small",
            creatureType = "humanoid",
            alignment = Alignment("neutral", "evil"),
            armorClass = 15,
            hitPointsMax = 7,
            speed = 30,
            abilityScores = AbilityScores(
                strength = 8,
                dexterity = 14,
                constitution = 10,
                intelligence = 10,
                wisdom = 8,
                charisma = 8
                ),
            proficiencyBonus = 2,
            challengeRating = 0.25,
            inventory = [
                instantiate_weapon_from_dict("scimitar"),
                GoldPieces(int(np.random.choice(np.arange(0, 4), p = [0.80, 0.09, 0.09, 0.02]))) # 80% chance of no gold, 9% chance each of 1 or 2 pieces and 2% chance of 3 pieces
                ],
            )
        self.equipWeapon(name = "scimitar", attackRollModifier = 4, damageDiceCount = 1, damageDiceSides = 6, damageRollModifier = 2)

    def __str__(self):
        return f"Goblin {self.name} smells blood!"


## CombatHandler class

In [354]:
class CombatHandler:
    """Class for handling combat mechanics"""
    def __init__(self, combatants):
        self.combatants = combatants
        self.activePlayerIndex = None
        self.activeTurnIndex = 0

    def introduceCombatants(self):
        """Introduce combatants in a random order"""
        random.shuffle(self.combatants)

        for combatant in self.combatants:
            print(combatant)
    
    def rollForInitiative(self):
        """All combatants roll for initiative"""
        for combatant in self.combatants:
            combatant.rollForInitiative()

        # Sort combatants list by initiative roll
        # Assumes no ties, need to implement tie resolver later
        self.combatants.sort(key = attrgetter("initiativeRollModified"), reverse = True)

        # Assign list index as initiativeRank attribute to lock in order of combat
        index = 0
        
        print("Roll for initiative:")
        
        for combatant in self.combatants:
            combatant.initiativeRank = index
            print(f'{combatant.name} rolled a {combatant.initiativeRollModified} and will go {int_to_ordinal(index + 1)}.')
            index += 1
        
        self.activePlayerIndex = 0
    
    def addCombatant(self, combatant):
        """Add a new combatant after combat begins"""
        if combatant in self.combatants:
            raise ValueError("Combatant is already fighting.")
        else:
            self.combatants.append(combatant)
            print(combatant)

    def removeCombatant(self, combatant):
        """Removes a combatant from play"""
        if combatant not in self.combatants:
            raise ValueError("Combatant is not in this fight.")
        else:
            self.combatants.remove(combatant)
            print(f'{combatant.name} has left the fight.')
        
        if len(self.combatants) == 1:
            print(f'{self.combatants[0].name} is the last one standing.')

    def incrementPlayerTurnCounter(self):
        """Increments active player and active turn indeces after a player is done taking actions for this turn"""
        if self.activePlayerIndex == len(self.combatants) - 1:
            self.activePlayerIndex = 0
            self.activeTurnIndex += 1
            print("A new turn is starting.")
        else:
            self.activePlayerIndex += 1
        
        print(f'{self.combatants[self.activePlayerIndex].name} is next.')

    def attackAction(self, target, weapon):
        """Resolves whether an attack hits, and how much damage it deals against the target. Then returns a message for flair"""
        attackDetails = self.combatants[self.activePlayerIndex].attack(target = target, weapon = weapon)

        if isinstance(weapon, Weapon):
            weaponName = weapon.name
        elif isinstance(weapon, str):
            weaponName = weapon
        else:
            raise ValueError("Weapon is neither Weapon object nor a weapon name.")
        
        targetStartingHitPoints = target.hitPoints

        if attackDetails.attackRoll == 1:
            message = "Womp womp it's a nat 1 and misses."
        elif attackDetails.attackRoll == 20:
            attackDetails.target.hitPoints = attackDetails.target.hitPoints - (attackDetails.damageRoll * 2)
            message = f'A critical hit dealing {attackDetails.damageRoll * 2} damage!'
        elif attackDetails.attackRollModified >= attackDetails.target.armorClass:
            attackDetails.target.hitPoints = attackDetails.target.hitPoints - attackDetails.damageRoll
            message = f'It hits, causing {attackDetails.damageRoll} damage.'
        elif attackDetails.attackRollModified < attackDetails.target.armorClass:
            message = "It misses."
        
        print(f'{self.combatants[self.activePlayerIndex].name} attacks {target.name} with {weaponName}.')
        print(f'{self.combatants[self.activePlayerIndex].name} rolls a {attackDetails.attackRoll} ({attackDetails.attackRollModified} with modifiers) against {target.name}\'s {target.armorClass} armor class.')
        print(message)
        print(f'{target.name} started with {targetStartingHitPoints} HP and now has {attackDetails.target.hitPoints}.')
        
        if attackDetails.target.hitPoints <= 0:
            print(f'{attackDetails.target.name} faints.')
        else:
            print(f'{attackDetails.target.name} is still standing.')
        
        self.incrementPlayerTurnCounter()


In [359]:
goblin1 = Goblin("Joe")

player1 = Player(name = "Ivan",
                 species = "human",
                 playerClass = "barbarian",
                 abilityScores = AbilityScores(16, 15, 14, 13, 12, 11),
                 proficiencyBonus = 3,
                 alignment = Alignment("lawful", "good"),
                 hitPointsMax = 30,
                 armorClass = 15,
                 weapons = [instantiate_weapon_from_dict("battle axe")]
                )

In [360]:
combat_obj = None # Reset
combat_obj = CombatHandler([player1, goblin1])
combat_obj.introduceCombatants()

Goblin Joe smells blood!
Barbarian Ivan jumps into battle!


In [361]:
combat_obj.rollForInitiative()

Roll for initiative:
Joe rolled a 9 and will go 1st.
Ivan rolled a 9 and will go 2nd.


In [365]:
# Use this for an Ivan attack
combat_obj.attackAction(target = goblin1, weapon = player1.weapons[0])

# Use this for a Goblin attack
#combat_obj.attackAction(target = player1, weapon = "scimitar")

Ivan attacks Joe with battle axe.
Ivan rolls a 19 (25 with modifiers) against Joe's 15 armor class.
It hits, causing 11 damage.
Joe started with 7 HP and now has -4.
Joe faints.
A new turn is starting.
Joe is next.


In [366]:
combat_obj.removeCombatant(goblin1)

Joe has left the fight.
Ivan is the last one standing.


## 10/19

Consider what happens here when n = 11

<font color="blue">
Peco: Fixed for 11, 12, and 13! Also changed num_to_ordinal() to be int_to_ordinal() for better clarity. Also added a new error catch.
</font>

## 10/19

I think there may be a misunderstanding or typo here:

```
for rolls in range(weapon.damageDiceCount):
        damage_dice_rolls.append(roll_dice(weapon.damageDiceSides))
        rolls += 1
```

You want to loop over the total number of weapon.damageDiceCount right? There is no need to increment `rolls`. In other words, you can remove the line `rolls += 1`. Also, I would call it `roll` instead of `rolls`. You are looping through each `roll` in that range. 

<font color="blue">
Peco: Good flag, the counter is not necessary at all. I think I got tripped up with how I've written some while loops in SQL.
</font>

---


I see that this code is repeated twice: 
```
damage_dice_rolls = []
    for rolls in range(weapon.damageDiceCount):
        damage_dice_rolls.append(roll_dice(weapon.damageDiceSides))
```
I know it's a small repetition, but you may want to make it into a function: `def get_damage_dice_rolls(total_rolls)`

<font color="blue">
Peco: Nice! I made a new get_multiple_dice_rolls() function because now that I think about it, this is a common loop that will be needed even beyond atatcks.
</font>

---

With this code:
```
    if weapon.weaponType == "melee":
        attack_and_damage_roll_modifier = ability_score_to_modifier(player.abilityScores.strength)
    elif weapon.weaponType == "ranged":
        attack_and_damage_roll_modifier = ability_score_to_modifier(player.abilityScores.dexterity)
```
There is a chance that `attack_and_damage_roll_modifier` never gets defined, if weaponType does not match one of those two choices. Consider what to do in this case. You can do this in an `else` block. This error could happen if you add a new weaponType in the future and forget to modify the player_attack function.

<font color="blue">
Peco: Good point, this covers all weapon types today but maybe in the future more types can get added. Added a new error catch here.
</font>

## 10/19

Nice checking for edge cases in `AbilityScores` and in `Alignment`. Maybe later we can talk about different ways to handle these, like throwing exceptions. This will ensure the code doesn't run and create a half-assed `Alignment` object. I don't think it is a critical thing to do though.

<font color="blue">
Peco: I made some minor tweaks to how errors are thrown, but sounds good we can maybe go over that later.
</font>

## 10/18


Kinda either super racist or super woke to give players a race. Have a story around this ready for HR.

<font color="blue">
Peco: Dude this was no kidding a big controversy in 2021. People were <b>extremely</b> upset that D&D isn't woke enough:
</font>

<url>https://www.wired.com/story/dandd-must-grapple-with-the-racism-in-fantasy/</url>

<font color="blue">
As a response they changed "race" to "species" in the new rules that just came out in September 2024 so that's a good flag for me to update it here too that way it matches up with the official rules.
</font>

<url>https://www.dndbeyond.com/posts/1393-moving-on-from-race-in-the-2024-core-rulebooks</url>

---
Why not make the `player_attack` function a method in `Player`?

<font color="blue">
Peco: Good call - done. When I was first writing out classes and functions I was figuring out whether there would be one attack function for both Player and Monster, or one for each class, and how many Player classes I should make. After I settled on a single Player class and different attack functions for Player and Monster I never cleaned this up.
</font>

## 10/19

Please add "sexy" to alignment here. And race should be "latinx".

<font color="blue">
Peco: Sorry but this doesn't seem to be working? Maybe Ivan can't be considered sexy (weird that it worked for other players when troubleshooting):

```
Alignment("sexy")

TypeError
Traceback (most recent call last)
Cell In[120], line 1
----> 1 Alignment("lawful", "good", "sexy")
```
</font>

## 10/19

Lets create a `GameHandler` class. All the logic below this comment should be implemented in that class.

<font color="blue">
Peco: Done for assignment 4!
</font>

## 10/19

You may want to consider putting the logic below this comment into your attack function itself.

<font color="blue">
Peco: Done! (It was the attackResolver logic)
</font>

## 10/19 Assignment 4

Address comments above, including creating a `GameHandler` class. Read up on 
Python inheritance [here](https://www.w3schools.com/python/python_inheritance.asp) and whatever other sources you find useful. Create a 
`Monster` parent class that all monster inherit from. Move your `monster_attack` function to the `Monster` parent class. Make a `Weapon` class 
and have your different weapons like `BattleAxe` and `Dildo` inherit from it. This should reduce the repetitiveness in code for the different weapons you are creating (they all have the exact same variables but are defined with different values).